## Given a control flow graph for Verilog code, how can we check if timing depends on a secret/sensitive variable?

If the code executes in constant time, our "done" signal should definitely not be tainted. There are some interesting additional cases to consider:
- what if execution is not constant time, but the timing does not depend on a secret?
- how do we handle loops in a control flow graph?

First, I'll consider the simple case where there are no loops in the control flow graph and not worry about whether edges of the graph depend on a secret or not. We can represent our graph as an adjacency list. Here is an example for the given graph A: 
![graph a](graph_a.png)

Note: Some code snippets were debugged/organized with assistance from chatGPT

In [8]:
graph_a = {
    'START': ['s1', 's2'],
    's1' : ['s3'],
    's2': ['s3'],
    's3': ['s4', 's5'],
    's4': ['s6'],
    's5': ['s6'],
    's6': ['END'],
    'END': []
}

There are two potential ways we can go about determining if execution is constant time. The first is by checking for nodes that dominate the END node (these are nodes that all paths go through during execution). Once we have identified "graph dominators", we can check if the path lengths from the previous dominator to the next dominator (the start and end nodes are trivially dominator nodes) are the same. Alternatively, we can just check if all the path lengths from the start to end node are the same.

Let's start by looking at the simpler case where we just check path lengths from the start to end node:

In [9]:
from collections import deque

def check_equal_path_lengths(graph, start, end):
    # queue of (node, length)
    queue = deque([(start, 0)])
    path_lengths = set()

    while queue:
        node, length = queue.popleft()

        # if we reach the end node, record the path length
        if node == end:
            path_lengths.add(length)
            continue

        # add neighbors to the queue with incremented length
        for neighbor in graph[node]:
            queue.append((neighbor, length + 1))

    # check if all path lengths are the same
    return len(path_lengths) == 1

To quickly test this, we can create another graph B that has paths with varying lengths:
![graph b](graph_b.png)

In [15]:
graph_b = {
    'START': ['s1', 's2'],
    's1' : ['s3'],
    's2': ['s3'],
    's3': ['s4', 's5'],
    's4': ['s7'],
    's5': ['s6'],
    's6': ['s7'],
    's7': ['END'],
    'END': []
}

In [16]:
print(check_equal_path_lengths(graph_a, 'START', 'END'))
print(check_equal_path_lengths(graph_b, 'START', 'END'))

True
False


These results match what we expect since the paths in graph A are all of equal lengths, so the function returns True. The same is not true in graph B, so the function returns false.

Now, let's consider the case where we only care if the execution time is affected by a variable that's designated secret. We can add additional information to our graph to indicate that an edge is dependent on a variable. One way to do this is to simply add a list of variable names to each node that indicate which variables affect the control flow "outgoing" from a particular node. This is a diagram that includes relevenant variables for state transitions along the edges.

![graph c](graph_c.png)

In [76]:
graph_c = {
    'START': [['s1', 's2'], ['secret', 'other_var']],
    's1' : [['s3'], []],
    's2': [['s3'], []],
    's3': [['s4', 's5'], ['secret', 'other_var']],
    's4': [['s7'], ['other_var']],
    's5': [['s6'], ['other_var']],
    's6': [['s7'], []],
    's7': [['END'], []],
    'END': [[],[]]
}

Now, we need to figure out how to check if differences in path length are dependent on a secret variable or not. We can do this by finding nodes that dominate the END node, and using these dominator nodes to count how many steps are affected by the secret.

Here is a more interesting example - a graph where not all path lengths are the same, but the path lengths do not depend on a secret variable:

![graph d](graph_d.png)

In [77]:
graph_d = {
    'START': [['s1', 's2'], ['secret', 'other_var']],
    's1' : [['s3'], []],
    's2': [['s3'], []],
    's3': [['s4', 's5'], ['other_var']],
    's4': [['s7'], ['other_var']],
    's5': [['s6'], ['other_var']],
    's6': [['s7'], []],
    's7': [['END'], []],
    'END': [[],[]]
}

One way to distinguish the timing dependency for these graphs is: 
- find the nodes that dominate the end node
- check if path lengths are equal between adjacent dominator nodes
- check if there is a dependency on a secret variable between adjacent dominator nodes
- if there is a secret dependency and path lengths are not equal between two dominators, return False (insecure)
- otherwise, return True (secure)

In [78]:
from collections import defaultdict
# compute the list of dominator nodes for the end node in order from start to end
def find_dominators(graph, start, end):
    # reverse the graph for dominator analysis
    reverse_graph = defaultdict(list)
    for node, (neighbors, _) in graph.items():
        for neighbor in neighbors:
            reverse_graph[neighbor].append(node)

    # initialize dominator sets
    all_nodes = set(graph.keys())
    dom = {node: set(all_nodes) for node in all_nodes}
    dom[start] = {start}

    # iteratively refine dominator sets
    changed = True
    while changed:
        changed = False
        for node in all_nodes - {start}:
            new_dom = set(all_nodes)
            for pred in reverse_graph[node]:
                new_dom &= dom[pred]
            new_dom.add(node)
            if new_dom != dom[node]:
                dom[node] = new_dom
                changed = True

    # extract dominators of the end node and sort them in order
    dominators_of_end = dom[end]
    ordered_dominators = []

    # perform BFS to order dominators from start to end
    visited = set()
    queue = deque([start])
    while queue:
        current = queue.popleft()
        if current in dominators_of_end and current not in visited:
            ordered_dominators.append(current)
            visited.add(current)
        neighbors = graph[current][0]  # Get neighbors
        for neighbor in neighbors:
            if neighbor not in visited:
                queue.append(neighbor)

    return ordered_dominators


In [71]:
from collections import deque

def check_equal_path_lengths_dependencies(graph, start, end, secret_vars):
    # queue of (node, length)
    queue = deque([(start, 0)]) 
    path_lengths = set()
    secret_dep = False

    while queue:
        node, length = queue.popleft()

        # if we reach the end node, record the path length
        if node == end:
            path_lengths.add(length)
            continue

        # access the neighbors
        neighbors = graph[node][0]
        for neighbor in neighbors:
            queue.append((neighbor, length + 1))

        # check for dependency on secret variable
        dependencies = graph[node][1]
        for dependency in dependencies:
            if dependency in secret_vars:
                secret_dep = True

    # check if all path lengths are the same
    return (len(path_lengths) == 1), secret_dep


In [79]:
def check_secret_dependency(graph, start, end, secret_vars):
    # find dominator nodes
    end_dominators = find_dominators(graph, start, end)

    # check if path lengths between dominators are equal 
    # if there are unequal paths between two dominators AND 
    # there is a secret dependency between those dominators, return false
    for i in range(len(end_dominators) - 1):
        equal_lengths, secret_dep = check_equal_path_lengths_dependencies(graph, end_dominators[i], end_dominators[i + 1], secret_vars)

        if not equal_lengths and secret_dep:
            return False
    return True


In [80]:
secret_vars = ['secret']
print(check_secret_dependency(graph_c, 'START', 'END', secret_vars))
print(check_secret_dependency(graph_d, 'START', 'END', secret_vars))

False
True


Insecure multiplier example:
![multiplier](mini_multiplier.png)

In [89]:
mini_multiplier = {
    'START': [['add1', 'shift1'], ['multiplier']],
    'add1' : [['shift1'], []],
    'shift1': [['add2', 'END'], ['multiplier']],
    'add2': [['END'], []],
    'END': [[],[]]
}

In [91]:
secret_vars = ['multiplier']
print(find_dominators(mini_multiplier, 'START', 'END'))
print(check_secret_dependency(mini_multiplier, 'START', 'END', secret_vars))

['START', 'shift1', 'END']
False


The multiplier is insecure as expected